# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [29]:
import requests
import csv
from bs4 import BeautifulSoup
l1=[i*10 for i in range(1,11)]
l2=[]
for i in l1:
  URL = 'https://citeseerx.ist.psu.edu/search?q=natural+language+processing&t=doc&sort=rlv&start={}'.format(i)
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  for i in soup.find_all("div",{"class":"snippet"}):
    l2.append(i.text)
import pandas as pd
df=pd.DataFrame({"Abstract":l2})
df.to_csv("mycsv1.csv")
df


,Abstract
0,"""... Introduction Statistical natural languag..."
1,"""... The paper summarizes the essential proper..."
2,"""... ABSTRACT: Language is way of communicatin..."
3,"""... We report experiments on the use of stand..."
4,"""... this paper, we will describe a simple rul..."
...,...
95,"""... 9. Bibliography ..."""
96,"""... Applying Natural Language Processing tech..."
97,"""... Examples of natural languages are Chinese..."
98,"""... A number of powerful modelling techniques..."


In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from textblob import Word
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [30]:
import pandas as pd
import re
import string

df = pd.read_csv("mycsv1.csv")

#remove special characters
df["Abstract_cln"] = df["Abstract"].str.replace(r"[^a-zA-Z ]+", " ").str.strip()

#remove punctuations
df['Abstract_cln'] = df['Abstract_cln'].str.replace('[^\w\s]','')

#remove numbers
df['Abstract_cln'] = df['Abstract_cln'].str.rstrip(string.digits)

#remove stopwords
stop = stopwords.words('english')
df['Abstract_cln'] = df['Abstract_cln'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#Lower case
df['Abstract_cln'] = df['Abstract_cln'].apply(lambda x: " ".join(x.lower() for x in x.split()))


#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['Abstract_cln'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


#Lemmatization
df['Abstract_cln'] = df['Abstract_cln'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [31]:
pos_tag=[]
frequency=[]
j=0
for i in df['Abstract_cln']:
  token=nltk.word_tokenize(i)
  pos_tag.append(nltk.pos_tag(token,tagset='universal'))
  frequency.append(nltk.FreqDist(tag for (word, tag) in pos_tag[0]))
  print(pos_tag[j])
  print(frequency[j])
  j=j+1
print(frequency)

[('introduction', 'NOUN'), ('statistical', 'ADJ'), ('natural', 'ADJ'), ('language', 'NOUN'), ('processing', 'NOUN'), ('snlp', 'ADJ'), ('field', 'NOUN'), ('lying', 'VERB'), ('intersection', 'ADJ'), ('natural', 'ADJ'), ('language', 'NOUN'), ('processing', 'NOUN'), ('machine', 'NOUN'), ('learning', 'VERB'), ('snlp', 'ADJ'), ('di', 'NOUN'), ('er', 'VERB'), ('traditional', 'ADJ'), ('natural', 'ADJ'), ('language', 'NOUN'), ('processing', 'NOUN'), ('instead', 'ADV'), ('linguist', 'ADJ'), ('manually', 'ADV'), ('construct', 'VERB'), ('model', 'NOUN'), ('given', 'VERB'), ('linguistic', 'ADJ')]
<FreqDist with 4 samples and 28 outcomes>
[('the', 'DET'), ('paper', 'NOUN'), ('summarizes', 'VERB'), ('essential', 'ADJ'), ('property', 'NOUN'), ('document', 'NOUN'), ('retrieval', 'NOUN'), ('review', 'VERB'), ('conventional', 'ADJ'), ('practice', 'NOUN'), ('research', 'NOUN'), ('finding', 'VERB'), ('latter', 'ADJ'), ('suggesting', 'VERB'), ('simple', 'ADJ'), ('statistical', 'ADJ'), ('technique', 'NOUN'),

In [7]:
!pip install benepar

     |████████████████████████████████| 3.2MB 8.5MB/s 
     |████████████████████████████████| 2.1MB 21.5MB/s 
     |████████████████████████████████| 1.2MB 31.5MB/s 
     |████████████████████████████████| 890kB 28.3MB/s 
  Created wheel for benepar: filename=benepar-0.2.0-cp37-none-any.whl size=37647 sha256=b681481b3164fdf671b347023e677de4bfa56e7347db5c844c3af440fcf9105d
  Stored in directory: /root/.cache/pip/wheels/dd/90/28/513063023646df7774be9401c440a5f13b48bdbab15a67fc42
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a80026f41a5d0b6acf4a4b24b7509d71aa2463a7616eab43503c3b856dd69155
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built benepar sacremoses


In [32]:
import spacy
nlp=spacy.load('en_core_web_sm')
from spacy import displacy
for i in df['Abstract_cln']:
  displacy.render(nlp(i),jupyter=True)

In [33]:
import spacy
from spacy import displacy
ner_dict=[]
nlp = spacy.load('en_core_web_sm')
for i in df['Abstract_cln']:
  for entity in nlp(i).ents:
    ner_dict.append((entity.label_,entity.text))
    print(entity.label_,entity.text)

ORG the paper summarizes essential property document retrieval review conventional practice research finding
ORG nlp
PERSON leake owen
ORG nlp
ORG nlp
ORG nlp
ORG nlp
ORG nlp
CARDINAL one
ORG nlp
LANGUAGE english
ORG nlp
PERSON automaton
PERSON automaton
ORG nlp
ORG nlp
ORG nlp
DATE four decade
ORG digital
ORG algorithm
DATE recent year
ORG nlp
ORG nlp
DATE past year
PERSON query
PERSON questionanswering qa technical
PERSON qa web
ORG qa
PERSON universit quot de saarlandes
ORG uni
GPE hamburg
ORG sri
DATE last five year
CARDINAL two
DATE early day
ORG nlp
DATE last year
GPE pa
ORG nlp
CARDINAL two
CARDINAL two
ORG nlp
ORG algorithm
CARDINAL two
ORG algorithm
CARDINAL two
ORG nlp
ORG nlp
CARDINAL three
ORG nlp
CARDINAL two
ORDINAL first
DATE last thirtyfive year
CARDINAL one three
ORG nlp
ORG nlp
ORG nlp
ORG nlp
ORDINAL first
ORDINAL second
ORG nlp
NORP indian
NORP indian
PERSON paul callaghan
PERSON durham degree
DATE august
LOC lolita lolita
NORP bigram
ORG nlp
DATE last thirtyfive ye

In [40]:
data=df.head(100)
data

,Unnamed: 0,Abstract,Abstract_cln
0,0,"""... Introduction Statistical natural languag...",introduction statistical natural language proc...
1,1,"""... The paper summarizes the essential proper...",the paper summarizes essential property docume...
2,2,"""... ABSTRACT: Language is way of communicatin...",abstract language way communicating word langu...
3,3,"""... We report experiments on the use of stand...",we report experiment use standard natural lang...
4,4,"""... this paper, we will describe a simple rul...",paper describe simple rule based approach auto...
...,...,...,...
95,95,"""... 9. Bibliography ...""",bibliography
96,96,"""... Applying Natural Language Processing tech...",applying natural language processing technique...
97,97,"""... Examples of natural languages are Chinese...",example natural language chinese english itali...
98,98,"""... A number of powerful modelling techniques...",a number powerful modelling technique develope...


In [41]:
data.to_csv('cleaned_data.csv')

In [42]:
import nltk
from nltk.util import ngrams
my_data = open('cleaned_data.csv', "r")
unigrams=ngrams(my_data.read().split(),1)
print(unigrams)
unigra_freq_dist=nltk.FreqDist(unigrams)

<generator object ngrams at 0x7f93d949eed0>


In [43]:
f = open('cleaned_data.csv')
raw = f.read()

tokens = nltk.word_tokenize(raw)

#Create your bigrams
bgs = nltk.bigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bgs)
for k,v in fdist.items():
    print(k,v)

Streaming output truncated to the last 5000 lines.
('new', 'opportunity') 1
('opportunity', 'challenge') 1
('challenge', 'presented') 1
('presented', 'ability') 1
('ability', 'search') 1
('full', '2,2') 1
('2,2', ',') 1
('...', 'ABSTRACT') 3
('ABSTRACT', ':') 2
(':', 'Language') 1
('Language', 'is') 1
('is', 'way') 1
('way', 'of') 1
('of', 'communicating') 1
('communicating', 'your') 1
('your', 'words') 1
('words', 'Language') 1
('Language', 'helps') 2
('helps', 'in') 1
('in', 'understanding') 1
('understanding', 'the') 1
('the', 'world') 3
('world', ',') 2
(',', 'we') 18
('we', 'get') 1
('get', 'a') 1
('a', 'better') 1
('better', 'insight') 2
('insight', 'of') 1
('of', 'the') 27
('world', '.') 2
('.', 'Language') 1
('helps', 'speakers') 1
('speakers', 'to') 1
('to', 'be') 6
('be', 'as') 1
('as', 'vague') 1
('vague', 'or') 1
('or', 'as') 1
('as', 'precise') 1
('precise', 'as') 1
('as', 'they') 2
('they', 'like') 1
('like', '.') 1
('.', 'NLP') 1
('NLP', 'Stands') 1
('Stands', 'for') 1
(

In [44]:
f = open('cleaned_data.csv')
raw = f.read()

tokens = nltk.word_tokenize(raw)

#bigrams
bgs = nltk.trigrams(tokens)

#frequency distribution for all the bigrams
fdist = nltk.FreqDist(bgs)
for k,v in fdist.items():
    print(k,v)

Streaming output truncated to the last 5000 lines.
('solving', 'different', 'kind') 1
('different', 'kind', 'complicated') 1
('kind', 'complicated', 'task') 1
('complicated', 'task', 'well') 1
('task', 'well', 'it') 1
('well', 'it', 'successfully') 1
('it', 'successfully', 'applied') 1
('successfully', 'applied', 'several') 1
('applied', 'several', 'field') 1
('several', 'field', 'image') 1
('field', 'image', '18,18') 1
('image', '18,18', ',') 1
('18,18', ',', "''") 1
('...', 'This', 'is') 1
('This', 'is', 'an') 1
('is', 'an', 'author-produced') 1
('an', 'author-produced', 'version') 1
('author-produced', 'version', 'of') 1
('version', 'of', 'a') 1
('of', 'a', 'paper') 1
('a', 'paper', 'published') 1
('paper', 'published', 'in') 1
('published', 'in', 'The') 1
('in', 'The', '...') 1
('The', '...', "''") 1
(',', 'this', 'author') 1
('this', 'author', 'produced') 1
('author', 'produced', 'version') 1
('produced', 'version', 'paper') 1
('version', 'paper', 'published') 1
('paper', 'publish

In [45]:
#trigram frequncies

clean_data = open('cleaned_data.csv', "r")
trigrams = ngrams(clean_data.read().split(), 3)
trigra_freq_dist=nltk.FreqDist(trigrams)
trigra_freq_dist

FreqDist({(',Unnamed:', '0,Abstract,Abstract_cln', '0,0,"""...'): 1,
          ('0,Abstract,Abstract_cln', '0,0,"""...', 'Introduction'): 1,
          ('0,0,"""...', 'Introduction', 'Statistical'): 1,
          ('Introduction', 'Statistical', 'natural'): 1,
          ('Statistical', 'natural', 'language'): 1,
          ('natural', 'language', 'processing'): 75,
          ('language', 'processing', '(SNLP)'): 1,
          ('processing', '(SNLP)', 'is'): 1,
          ('(SNLP)', 'is', 'a'): 1,
          ('is', 'a', 'field'): 1,
          ('a', 'field', 'lying'): 1,
          ('field', 'lying', 'in'): 1,
          ('lying', 'in', 'the'): 1,
          ('in', 'the', 'intersection'): 1,
          ('the', 'intersection', 'of'): 1,
          ('intersection', 'of', 'natural'): 1,
          ('of', 'natural', 'language'): 8,
          ('language', 'processing', 'and'): 1,
          ('processing', 'and', 'machine'): 1,
          ('and', 'machine', 'learning.'): 1,
          ('machine', 'learning.',

In [46]:
clean_data = open('cleaned_data.csv', "r")
bigrams = ngrams(clean_data.read().split(), 2)
bigra_freq_dist=nltk.FreqDist(bigrams)

In [47]:
#probability of  all the bigrams
def probability_bigram():
  for bigram_values in bigra_freq_dist:
#    print(bigram_values)
    print((" The bigram values", bigram_values,"Probability :",bigra_freq_dist[bigram_values]/unigra_freq_dist[(bigram_values[0],)]))

probability_bigram()

(' The bigram values', (',Unnamed:', '0,Abstract,Abstract_cln'), 'Probability :', 1.0)
(' The bigram values', ('0,Abstract,Abstract_cln', '0,0,"""...'), 'Probability :', 1.0)
(' The bigram values', ('0,0,"""...', 'Introduction'), 'Probability :', 1.0)
(' The bigram values', ('Introduction', 'Statistical'), 'Probability :', 0.2)
(' The bigram values', ('Statistical', 'natural'), 'Probability :', 1.0)
(' The bigram values', ('natural', 'language'), 'Probability :', 0.9051724137931034)
(' The bigram values', ('language', 'processing'), 'Probability :', 0.6)
(' The bigram values', ('processing', '(SNLP)'), 'Probability :', 0.010101010101010102)
(' The bigram values', ('(SNLP)', 'is'), 'Probability :', 1.0)
(' The bigram values', ('is', 'a'), 'Probability :', 0.15517241379310345)
(' The bigram values', ('a', 'field'), 'Probability :', 0.00980392156862745)
(' The bigram values', ('field', 'lying'), 'Probability :', 0.18181818181818182)
(' The bigram values', ('lying', 'in'), 'Probability :',

In [49]:
import nltk
nltk.download('averaged_perceptron_tagger')
from textblob import TextBlob
clean_data = open('cleaned_data.csv', "r")

_noun = lambda pos: pos[:2] == 'NN'
tokenized = nltk.word_tokenize(clean_data.read())
nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if _noun(pos)] 

print("nouns:\n",nouns)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
nouns:
 ['Abstract', 'Abstract_cln', 'Introduction', 'Statistical', 'language', 'processing', 'SNLP', 'field', 'intersection', 'language', 'processing', 'machine', 'learning', 'SNLP', 'ers', 'language', 'processing', 'linguist', 'model', 'linguistic', 'introduction', 'language', 'processing', 'field', 'language', 'processing', 'machine', 'di', 'language', 'processing', 'model', 'paper', 'properties', 'document', 'retrieval', 'practice', 'research', 'findings', 'suggesting', 'techniques', 'opportunities', 'challenges', 'ability', 'paper', 'property', 'document', 'retrieval', 'practice', 'research', 'technique', 'opportunity', 'challenge', 'ability', 'search', 'ABSTRACT', 'Language', 'way', 'words', 'Language', 'world', 'insight', 'world', 'Language', 'speakers', 'NLP', 'Stands', 'language', 'Natural', 

In [51]:
import nltk
nltk.download('brown')
clean_data = open('cleaned_data.csv', "r")

blob = TextBlob(clean_data.read())
noun_phrases = blob.noun_phrases
print("nouns phrases:\n",noun_phrases) 

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
nouns phrases:
 ['unnamed', 'abstract', 'abstract_cln', 'introduction statistical', 'natural language processing', 'snlp', 'natural language processing', 'machine learning', 'snlp', 'di # ers', 'traditional natural language processing', 'linguistic ...', 'statistical natural language processing snlp field', 'natural language processing machine learning snlp di er', 'traditional natural language processing', 'construct model', 'paper summarizes', 'essential properties', 'document retrieval', 'conventional practice', 'research findings', 'simple statistical techniques', 'new opportunities', 'full ...', 'paper summarizes', 'essential property document retrieval review', 'conventional practice research', 'simple statistical technique', 'new opportunity challenge', 'ability search', 'abstract', 'language', 'language', 'language', 'nlp stands', 'natural language processing..', 'natu

In [52]:
df_noun = pd.DataFrame(noun_phrases, columns = ['noun_phrases'])
df_noun 

,noun_phrases
0,unnamed
1,abstract
2,abstract_cln
3,introduction statistical
4,natural language processing
...,...
1111,approa ...
1112,approach design database
1113,erd notation
1114,stage database development stage user requirem...


In [53]:
clean_data = open('cleaned_data.csv', "r")
f = nltk.FreqDist(noun_phrases)
f 

FreqDist({'# programs': 1,
          '... ...': 6,
          '... algorithms': 1,
          '... research': 1,
          '... uni-hamburg.de ...': 1,
          '... visual development environment': 1,
          '... work': 1,
          'ability search': 1,
          'abstract': 12,
          'abstract diff software program detects difference': 1,
          'abstract language way': 1,
          'abstract metabolism machinery life signal transduction': 1,
          'abstract-': 1,
          'abstract-a': 1,
          'abstract_cln': 1,
          'abstract—natural': 1,
          'abstract—natural language processing': 1,
          'account meaning extraction pr': 1,
          'acquaintance principle': 1,
          'acquisition process': 1,
          'acquisition process ...': 1,
          'adaptive model': 1,
          'adaptive models': 1,
          'aid biologist': 1,
          'aid machine translation speech recognition w': 1,
          'alternative resource nlp task': 1,
          'al

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [54]:
my_tf_result = (df['Abstract_cln']).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
my_tf_result.columns = ['Abstract_words','tf']
my_tf_result

,Abstract_words,tf
0,processing,62.0
1,language,89.0
2,natural,74.0
3,snlp,2.0
4,statistical,5.0
...,...,...
1024,semi,1.0
1025,notation,1.0
1026,approa,1.0
1027,determination,1.0


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA

In [55]:
tn_set = df['Abstract_cln'].values.tolist() #train set
my_tt_set = "introduction statistical natural language processing snlp field lying intersection natural language processing machine learning snlp di er traditional natural language processing " #Query taken from one of the tweets which is called test set
my_tt_set = [my_tt_set]
stopWords = stopwords.words('english')
vzer = CountVectorizer(stop_words = stopWords)

transformer = TfidfTransformer()

In [56]:
tnVectorizerArray = vzer.fit_transform(tn_set).toarray()
ttVectorizerArray = vzer.transform(my_tt_set).toarray()
cx = lambda a, b : np.inner(a, b)/(LA.norm(a)*LA.norm(b))
cosine_values = []

In [57]:
for vector in tnVectorizerArray:
        for testV in ttVectorizerArray:
            cosine = cx(vector, testV)
            cosine_values.append(cosine)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
df_result = df.filter(['Abstract_cln'], axis=1)
se = pd.Series(cosine_values)
df_result['Cosine_similarity'] = se.values
df_result

,Abstract_cln,Cosine_similarity
0,introduction statistical natural language proc...,0.924211
1,the paper summarizes essential property docume...,0.031235
2,abstract language way communicating word langu...,0.527086
3,we report experiment use standard natural lang...,0.225070
4,paper describe simple rule based approach auto...,0.029001
...,...,...
95,bibliography,0.000000
96,applying natural language processing technique...,0.252447
97,example natural language chinese english itali...,0.468521
98,a number powerful modelling technique develope...,0.163118


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# uploaded the csv file

 